In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn

from torch.utils.data import DataLoader
from Dataset.WindowDataset import WindowDataset

from Model.AE import Auto
from Model.LSTM import LSTM
from Model.LODE2 import ODEVAE

from utils import combine_tensors

In [2]:
eco = '콜금리'
batch_size = 4
max_apart_size = 70  # 0~60평 아파트만 돌림
real_estate_weighted_average = 0.5
input_size = 5 # LSTM input_size
hidden_size = 16 # LSTM hidden_size

In [3]:
def make_estimate(x):
    vec = np.zeros((max_apart_size))
    vec[int(x[0])] = x[1]
    return vec

In [35]:
def get_date(x):
    date = str(x[1])
    if len(date)==1:
        date = '0'+date
    return str(x[0])+date

In [39]:
df = pd.read_csv('../데이터/merge.csv')
df['시점'] = df[['계약년월','계약일']].apply(get_date, axis=1)
df = df[:100] # /////////////////////////////////
df['시점'] = df['시점'].apply(lambda x : int(x))
df['평수'] = df['전용면적(㎡)'].apply(lambda x : x // 3.3058)
df['가격'] = df['거래금액(만원)'].apply(lambda x : int(x.replace(',','')))
df = df.sort_values(by=['시점'])
df['부동산'] = df[['평수','가격']].apply(make_estimate,axis=1)
df.reset_index(drop=True)
df['경제'] = df[eco]
df = df[['시점','부동산','경제']]
df = df.reset_index(drop=True)

In [40]:
len_df = len(df)
train_len = int(len_df *0.6)
val_len = int(len_df * 0.3)

batch_size = 4

train_dataset = WindowDataset(df[:train_len])
train_loader = DataLoader(train_dataset, batch_size=batch_size)

val_dataset = WindowDataset(df[train_len:train_len+val_len])
val_loader = DataLoader(val_dataset, batch_size=batch_size)

test_dataset = WindowDataset(df[train_len+val_len:])
test_loader = DataLoader(test_dataset, batch_size=batch_size)

In [20]:
# x1 : 시점/경제 데이터, x2 : 부동산 데이터
# y1 : 시점/경제 라벨링, y2 : 부동산 라벨링

for x1,x2,y1,y2 in train_loader:
    print(x1.shape)
    print(x2.shape)
    print(y1.shape)
    print(y2.shape)
    break

torch.Size([4, 5, 2])
torch.Size([4, 5, 70])
torch.Size([4, 2])
torch.Size([4, 70])


In [17]:
# 모델 입출력 크기

input_size = 256
window_size = 5
hidden_size = 512
latent_size = 512

In [43]:
######### 모델
Model = ODEVAE(input_size,hidden_size,latent_size)

model = Model.to(device)

print('작동하는지 실험')
basic_data = torch.rand((window_size,1,input_size))
t = torch.rand((window_size,1,1))
model(basic_data,t)

작동하는지 실험


(tensor([[[-0.1136, -0.3061,  0.4087,  ...,  0.4813,  0.2837, -0.4833]],
 
         [[-0.1697, -0.2385,  0.3640,  ...,  0.5128,  0.2821, -0.4445]],
 
         [[-0.1436, -0.2685,  0.3844,  ...,  0.4979,  0.2829, -0.4623]],
 
         [[-0.1119, -0.3078,  0.4099,  ...,  0.4802,  0.2837, -0.4844]],
 
         [[-0.1570, -0.2523,  0.3737,  ...,  0.5054,  0.2825, -0.4530]]],
        grad_fn=<ViewBackward0>),
 tensor([[-4.8150e-01,  1.1618e+00,  3.8211e-01, -1.8602e+00,  2.0590e+00,
          -9.4056e-01,  2.6450e-01, -1.0925e+00,  4.9431e-01, -1.7121e+00,
          -1.6731e-01,  1.7961e-01, -1.1171e+00,  2.3820e-01,  1.0659e+00,
          -1.8009e+00,  8.3070e-01,  1.0767e+00,  1.1459e+00,  1.1616e+00,
          -6.9344e-01, -3.9435e-01,  2.1104e+00, -2.0717e-01, -5.6266e-01,
           7.2708e-01,  6.5230e-01, -1.0362e+00, -4.3800e-01,  1.5745e+00,
          -1.1096e+00,  1.3129e+00, -7.2656e-01,  1.7568e+00,  2.1071e-02,
           1.2842e+00,  4.9609e-01,  1.0825e+00, -2.9317e-02,  1.37

In [12]:
#############################
# 데이터 & 모델에 device 붙임!!!
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(f'{device} is available')

# model = Model.to(device)

# print('작동하는지 실험')
# basic_data = torch.rand((1,input_size))
# model(basic_data)

criterion = nn.MSELoss()

lr = 1e-3
num_epochs = 10
noise_std = 0.02
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

cpu is available


In [13]:
economy_embedding_model = LSTM(input_size,hidden_size).to(device)
economy_embedding_model.load_state_dict(torch.load('Checkpoint/economic_best_model.pth'))

real_estate_embedding_model = Auto(max_apart_size).to(device)
real_estate_embedding_model.load_state_dict(torch.load('Checkpoint/embedding_best_model.pth'))

RuntimeError: Error(s) in loading state_dict for Auto:
	Unexpected key(s) in state_dict: "encoder.4.weight", "encoder.4.bias", "decoder.4.weight", "decoder.4.bias". 
	size mismatch for encoder.0.weight: copying a param with shape torch.Size([64, 7]) from checkpoint, the shape in current model is torch.Size([32, 70]).
	size mismatch for encoder.0.bias: copying a param with shape torch.Size([64]) from checkpoint, the shape in current model is torch.Size([32]).
	size mismatch for encoder.2.weight: copying a param with shape torch.Size([64, 64]) from checkpoint, the shape in current model is torch.Size([16, 32]).
	size mismatch for encoder.2.bias: copying a param with shape torch.Size([64]) from checkpoint, the shape in current model is torch.Size([16]).
	size mismatch for decoder.0.weight: copying a param with shape torch.Size([64, 256]) from checkpoint, the shape in current model is torch.Size([32, 16]).
	size mismatch for decoder.0.bias: copying a param with shape torch.Size([64]) from 

In [ ]:
for epoch in range(num_epochs):
    print('epoch : {}'.format(epoch))
    running_loss = 0.0
    num = 0
    print("train")
    model.train()
    for x1, x2, y1, y2 in train_loader:
        x1, x2 = x1.to(device), x2.to(device) # x1: [시점, 경제], x2: 부동산
        y1, y2 = y1.to(device), y2.to(device)
        
        time_data = x1[:,:,0]
        eco_data = x1[:,:,1]
        
        # 경제 데이터
        eco_emb = economy_embedding_model(eco_data)
        # 부동산 데이터
        real_est_emb = real_estate_embedding_model(x2)
        
        final_emb = combine_tensors(real_est_emb, eco_emb, real_estate_weighted_average, 'sum')
        ##########################
        # 모델에 x[0], final_emb 가지고 ODE 학습하기!!!!
        
        optimizer.zero_grad()

        x_p, z, z_mean, z_log_var = model(final_emb, time_data)
        kl_loss = -0.5 * torch.sum(1 + z_log_var - z_mean**2 - torch.exp(z_log_var), -1)
        loss = 0.5 * ((final_emb-x_p)**2).sum(-1).sum(0) / noise_std**2 + kl_loss
        loss = torch.mean(loss)
       
        loss.backward()
        optimizer.step()